In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2022/11/08 15:54
# @Author  : Wang Yujia
# @File    : basicinfo_GT_models.ipynb
# @Description : 关于GT模型的计算结果的基本信息

# 0. What for
1. 关于GT模型的计算结果的基本信息

# 1. Preparations
## 1.1 全局设置

In [2]:
# P in info_insym
P_1_path = "../data/info_asymm/results/asc_symmetry/GT_asc_symmetry_2_P.csv"
P_2_path = "../data/pt/results/PT_all1288_P.csv"

features_GT = ['product_id','bidincrement','bidfee','retail']

import numpy as np
import csv
import pandas as pd
from visdom import Visdom

# viz = Visdom(env='P',use_incoming_socket=False)

## 1.2 读取data


In [3]:
P_1 = pd.read_csv(P_1_path, encoding="utf-8")
P_2 = pd.read_csv(P_2_path, encoding="utf-8")

# 2. Info about P which is the input of NNs
## 2.1 How many auction settings are included in 'info_insymm', 'GT'

In [8]:
print(f"In 'info_insymm', there is *{P_1.shape}*")
print(f"In 'GT', there is *{P_2.shape}*")

In 'info_insymm', there is *(0, 612)*
In 'GT', there is *(0, 1288)*
